Download from here https://finance.google.com/finance/historical?q=NASDAQ:GOOG

In [4]:
import locale
locale.getdefaultlocale()

ValueError: unknown locale: UTF-8

In [3]:
### Load in necessary libraries for data input and normalization
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

### load in and normalize the dataset
dataset = np.loadtxt('goog.csv')

ValueError: unknown locale: UTF-8